In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import json


In [43]:
# Đọc dữ liệu đã xử lý
data = pd.read_csv('../data/processed/diamonds_clean.csv')

# Đọc thông tin folds
with open('../data/kfold_indices.json', 'r') as f:
    fold_indices = json.load(f)


In [44]:
# Giả sử cột target là 'price'
X = data.drop(columns=['price'])
y = data['price']

print("Shape dữ liệu:", X.shape, y.shape)


Shape dữ liệu: (53794, 9) (53794,)


In [45]:
def cross_val_mae_sklearn(X, y, model, k=5, random_state=42):
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    maes = []
    
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Sklearn tự động xử lý intercept (hệ số chặn)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        
        # Sử dụng MAE của sklearn
        maes.append(mean_absolute_error(y_val, y_pred))
    
    return np.mean(maes)

In [46]:
model_definitions = [
    # Minh
    {"Tên mô hình": "Model 1: carat", 
     "Đặc trưng": ['carat']},

    # My
    {"Tên mô hình": "Model 2: carat + depth + table", 
     "Đặc trưng": ['carat', 'depth', 'table']},

    # Kiệt
    {"Tên mô hình": "Model 3: carat + x + y + z", 
     "Đặc trưng": ['carat', 'x', 'y', 'z']},

    # Thảo 
    {"Tên mô hình": "Model 4: carat + depth + table + x + y + z", 
     "Đặc trưng": ['carat', 'depth', 'table', 'x', 'y', 'z']},

    # Duy 
    {"Tên mô hình": "Model 4 (Duy): carat + depth + table + x + y + z", 
     "Đặc trưng": ['carat', 'depth', 'table', 'x', 'y', 'z']}
]

In [47]:
results = []

for m in model_definitions:
    cols = m["Đặc trưng"]
    
    # Khởi tạo mô hình sklearn
    model = LinearRegression()
    
    # Sử dụng hàm cross_val_mae_sklearn
    mae_score = cross_val_mae_sklearn(X[cols], y, model, k=5)
    
    print(f"Hoàn thành: {m['Tên mô hình']}")
    
    results.append({
        "Tên mô hình": m["Tên mô hình"],
        "Đặc trưng": cols,
        "MAE trung bình": mae_score
    })

Hoàn thành: Model 1: carat
Hoàn thành: Model 2: carat + depth + table
Hoàn thành: Model 3: carat + x + y + z
Hoàn thành: Model 4: carat + depth + table + x + y + z
Hoàn thành: Model 4 (Duy): carat + depth + table + x + y + z


In [48]:
results_df = pd.DataFrame(results)
results_df.sort_values("MAE trung bình")

,Tên mô hình,Đặc trưng,MAE trung bình
3,Model 4: carat + depth + table + x + y + z,"[carat, depth, table, x, y, z]",888.079178
4,Model 4 (Duy): carat + depth + table + x + y + z,"[carat, depth, table, x, y, z]",888.079178
2,Model 3: carat + x + y + z,"[carat, x, y, z]",905.567051
1,Model 2: carat + depth + table,"[carat, depth, table]",994.625593
0,Model 1: carat,[carat],1007.743775
